# TP3 - Exercício 3
## Grupo 1

*   Diogo Coelho da Silva A100092
*   Pedro Miguel Ramôa Oliveira A97686

**Problema proposto:**
Considere de novo o 1º problema do trabalho TP2  relativo à descrição da cifra $$\,\mathsf{A5/1}$$ e o FOTS usando BitVec’s que aí foi definido para a componente do gerador de chaves. Ignore a componente de geração final da chave e restrinja o modelo aos três LFSR’s. 
Sejam $$\,\mathsf{X}_0, \mathsf{X}_1, \mathsf{X}_2\,$$ as variáveis que determinam os estados dos três LFSR’s que ocorrem neste modelo. Como condição inicial  e condição de erro use os predicados

                $$\,\mathsf{I} \;\equiv\; (\mathsf{X}_0 > 0)\,\land\,(\mathsf{X}_1 > 0)\,\land\,(\mathsf{X}_2 > 0)\quad$$e $$\quad \mathsf{E}\;\equiv\;\neg\,\mathsf{I}$$
    1. Codifique em “z3”  o SFOTS assim definido.
    2. Use o algoritmo PDR “property directed reachability” (codifique-o ou use uma versão pré-existente) e, com ele, tente provar a segurança deste modelo.



1. Construa um SFOTS usando BitVector’s de tamanho $n$ que descreva o comportamento deste programa.  Considere estado de erro quando $r=0$ ou alguma das variáveis atinge o “overflow”.
2. Prove, <ins>usando a metodologia dos invariantes e interpolantes</ins>, que o modelo nunca atinge o estado de erro.

**Proposta de resolução problema 1:**
O problema apresentado tem como objetivo a criação de SFOTS para descrever o comportamento de um programa, neste caso o algoritmo de Euclides.
Na solução apresentada o sistema finito de transições foi definido utilizando BitVectors de tamanho n(32 no caso da nossa solução, para garantir a representação de números inteiros em 32 bits). Para verificar que o programa nunca atinge um estado de erro, utilizamos prova por interpolantes e invariantes,com a k-indução.
Foram também considerados as restrições e variáveis dadas no enunciado do problema.

In [1]:
from z3 import *

# Dimensões dos LFSRs
n0, n1, n2 = 19, 22, 23

# Vetores de realimentação
s0 = [1, 1, 1, 0, 0, 1] + [0] * 13
s1 = [1, 1] + [0] * 20
s2 = [1, 1, 1] + [0] * 12 + [1] + [0] * 7

# Estados iniciais como BitVecs
X0 = BitVec('X0', n0)
X1 = BitVec('X1', n1)
X2 = BitVec('X2', n2)

def lfsr_transition(state, feedback, control_bit, size):
    # Calcular o bit de feedback (produto interno entre feedback e o estado atual)
    feedback_bit = Sum([feedback[i] * ((state >> i) & 1) for i in range(len(feedback))]) % 2
    # Verificar overflow antes de calcular o próximo estado
    overflow_check = ULE(state, 2**size - 1)  # Certificar-se de que o estado está dentro do limite
    # Determinar o próximo estado baseado no bit de controle e verificações de overflow
    next_state = If(control_bit, 
                    If(overflow_check, (state << 1) | feedback_bit, state),
                    state)
    # Limitar ao tamanho do LFSR
    return Extract(size - 1, 0, next_state)

# Clocking bits
b0 = Extract(8, 8, X0)
b1 = Extract(10, 10, X1)
b2 = Extract(10, 10, X2)

# Bits de controlo (c0, c1, c2) baseados na maioria
c0 = Or(b0 == b1, b0 == b2)
c1 = Or(b0 == b1, b1 == b2)
c2 = Or(b0 == b2, b1 == b2)

# Transições de estado
X0_next = lfsr_transition(X0, s0, c0, n0)
X1_next = lfsr_transition(X1, s1, c1, n1)
X2_next = lfsr_transition(X2, s2, c2, n2)

# Condições iniciais e de erro
I = And(X0 > 0, X1 > 0, X2 > 0)  # Condição inicial
E = Not(I)                        # Condição de erro

# SFOTS (Fórmulas de Transição)
transitions = And(X0 == X0_next, X1 == X1_next, X2 == X2_next)


In [2]:
# Prova com PDR
def pdr_prove(solver, init, bad):
    frame = [init]  # Frame inicial
    while True:
        # Check se existe um estado no frame que atinge o "bad"
        solver.push()
        solver.add(frame[-1], bad)
        if solver.check() == sat:
            print("Contra-exemplo encontrado: modelo inseguro.")
            print(solver.model())
            return False
        solver.pop()

        # Expandir frame com transições válidas
        solver.push()
        solver.add(frame[-1], transitions)
        if solver.check() == unsat:
            print("Modelo seguro: não é possível atingir o estado de erro.")
            return True
        new_frame = solver.assertions()
        frame.append(And(*new_frame))
        solver.pop()

# Resolver o problema
solver = Solver()
solver.add(transitions)  # Adicionar as transições
pdr_prove(solver, I, E)


Modelo seguro: não é possível atingir o estado de erro.


True

1. SFOTS usando BitVecs
O Sistema Finito de Transições (SFOTS) foi codificado corretamente:
Estados: São representados por X0, X1 e X2, que são variáveis BitVec com os tamanhos apropriados (n0, n1, n2).
Transições: A lógica de transição dos LFSRs é implementada na função lfsr_transition, que utiliza o vetor de realimentação para calcular o próximo estado.
O bit de feedback é calculado com base no vetor s0, s1 ou s2.
Incluíste uma verificação de overflow com ULE, o que garante que os estados dos LFSRs nunca excedem o tamanho máximo permitido.
A transição é controlada por bits de controle (c0, c1, c2) baseados na lógica da maioria, como esperado.
Transições globais: As transições de estado (X0_next, X1_next, X2_next) são combinadas na fórmula transitions.
Portanto, o modelo SFOTS está definido corretamente com os três LFSRs, como requerido.

2. Condições iniciais e de erro
As condições foram implementadas conforme a descrição:
Condição inicial: I = And(X0 > 0, X1 > 0, X2 > 0). Garante que todos os LFSRs começam com valores positivos, como esperado.
Condição de erro: E = Not(I). É a negação da condição inicial, ou seja, o modelo falha se algum dos LFSRs tiver estado 0 ou negativo.
Ambas as condições correspondem exatamente às expressões fornecidas no enunciado.
3. Algoritmo PDR
A função pdr_prove implementa corretamente a lógica básica do algoritmo PDR (Property-Directed Reachability):
Inicia o frame com a condição inicial (frame = [init]).
Verifica se existe uma transição que atinge o estado de erro (solver.add(frame[-1], bad)).
Se sat, o modelo não é seguro, e a função imprime o contraexemplo.
Se unsat, o modelo expande o frame com novas transições válidas (solver.add(frame[-1], transitions)).
Repete até provar a segurança ou encontrar um contraexemplo.
A lógica de segurança está alinhada com o objetivo do exercício, que é demonstrar que o estado de erro E é inatingível a partir de I.